In [5]:
% LSG-CPD: CPD with Local Surface Geometry
% Demo of Multi-view experiments
% Author: Weixiao Liu, Hongtao Wu 
% Johns Hopkins University & National University of Singapore

% Armadillo - 12 views
% Dragon    - 15 views
% Happy     - 15 views

clear
clc
close all
%addpath generate working directory
addpath(genpath(pwd));

In [16]:
%字符串比对，实现
model_name = "Bunny";
num_pc=0;
if strcmp(model_name, 'Happy') || strcmp(model_name, 'dragon')
    num_pc = 2;
else
    if strcmp(model_name, 'Armadillo')
        num_pc = 12;
    
    else
        if strcmp(model_name, 'Bunny')
            num_pc=10;
        end
    end     
end

In [17]:
% --------------- Load data ---------------
path = strcat("data/multiview/", model_name, '/');
pc = cell(1, num_pc);

for i = 1 : num_pc
    filename = sprintf("%d.ply", i);
    pc{i} = pcread(strcat(path, filename));
end

# 1,strcat函数即字符串拼接函数
# 2，pcread函数，点云读取
## 1，ptCloud = pcread(filename) 从PLY、PCD读取指定路径的点云。反回一个pointCloud对象。
## 2，pointcloud对象包含属性
    1. Location——3D点坐标，数据格式为M*3矩阵或者M*N*3矩阵
    2. Color——3D点的RGB颜色 信息，数据格式同上
    3. Normal——3D点的法向量信息，数据格式同上
    4. Count——3D点的数量
    5. Xlimits——X坐标大小范围
    6. Ylimits——Y坐标大小范围
    7. Zlimits——Z坐标大小范围
    
# 3,点云存储
~~~matlab
 pcwrite(ptCloud,filename)
 pcwrite(ptCloud,filename,'Encoding',encodingType)
~~~
 # 4，点云显示
     pcshow
   
 

In [ ]:
% --------------- Initial point cloud ---------------
[pc_merge] = MergePointClouds(pc, 'pointCloud');
% 点云拼接函数
figure
ShowPointClouds(pc_merge, 'backgroundColor', 'white', 'grid', 'hide', 'axis', 'hide', 'dataset', model_name);
title('Randomly Initialized Point Clouds')

xform_rel = cell(1, num_pc-1); % relative xform
xform_abs = cell(1, num_pc);   % absolute xform

for i = 1 : num_pc-1
    xform_rel{i} = LSGCPD(pc{i+1}, pc{i}, 'outlierRatio', 0.9);
end

xform_abs{1} = eye(4);
for i = 1 : num_pc-1
    temp_xform_rel = xform_rel{i};
    xform_abs{i+1} = xform_abs{i} * [temp_xform_rel.Rotation', temp_xform_rel.Translation'; 0 0 0 1];
end

In [ ]:
pc_result = cell(1, num_pc);
pc_result{1} = pc{1};
for i = 2 : num_pc
    temp_xform_abs = rigid3d(xform_abs{i}(1:3, 1:3)', xform_abs{i}(1:3, 4)');
    pc_result{i} = pctransform(pc{i}, temp_xform_abs);
end

pc_result_merge = MergePointClouds(pc_result, 'pointCloud');
% pc_result_merge = pcdenoise(pc_result_merge, 'Threshold', 1, 'NumNeighbors', 50);
figure
disp(pc_result_merge.Count)
ShowPointClouds(pc_result_merge, 'backgroundColor', 'white', 'grid', 'hide', 'axis', 'hide', 'dataset', model_name);

In [ ]:
for i = 1 : 10
   disp(xform_abs{i}.Rotation)
end

In [ ]:
for i=1:num_pc-1
    disp(xform_abs{i})
end

In [ ]:
disp(xform_abs{2})

In [ ]:
disp(xform_abs{3})